---
jupyter:
  jupytext:
    formats: md,ipynb
    text_representation:
      extension: .md
      format_name: markdown
      format_version: '1.3'
      jupytext_version: 1.16.0
  kernelspec:
    display_name: Python 3 (ipykernel)
    language: python
    name: python3
---

<!-- #region id="b40a2694" -->
# Table des matières
1. [Importance de la normalisation des données](#importance-de-la-normalisation-des-données)
1. [Rééchelonnage ou normalisation](#rééchelonnage-ou-normalisation)
    1. [Lecture de la base de données](#lecture-de-la-base-de-données)
    1. [Méthodes de transformation](#méthodes-de-transformation)
      1. [Données brutes](#données-brutes)
      1. [Standard](#standard)
      1. [Min-max](#min-max)
      1. [Max-abs](#max-abs)
      1. [Robuste (basé sur les quantiles 1, 2 et 3)](#robuste-basé-sur-les-quantiles-1-2-et-3)
      1. [Transformation par puissance](#transformation-par-puissance)
      1. [Transformation quantile à quantile](#transformation-quantile-à-quantile)
      1. [Normalisation vectorielle](#normalisation-vectorielle)
    1. [Comparaison entre les différents rééchantillonneurs](#comparaison-entre-les-différents-rééchantillonneurs)
1. [Le traitement des variables catégorielles](#le-traitement-des-variables-catégorielles)
  1. [Regroupement de catégories en une seule valeur](#regroupement-de-catégories-en-une-seule-valeur)
  1. [Regroupement de variables numériques en catégories](#regroupement-de-variables-numériques-en-catégories)
    1. [Catégories de taille égales](#catégories-de-taille-égales)
    1. [Catégories par intervalles identiques](#catégories-par-intervalles-identiques)
  1. [Encoder des variables catégorielles](#encoder-des-variables-catégorielles)
    1. [Encoder une variable booléenne en la transformant en entier](#encoder-une-variable-booléenne-en-la-transformant-en-entier)
    1. [Encoder manuellement en établissant les correspondances avec un dictionnaire](#encoder-manuellement-en-établissant-les-correspondances-avec-un-dictionnaire)
    1. [Encoder automatiquement](#encoder-automatiquement)
<!-- #endregion -->



In [ ]:
%matplotlib inline

import matplotlib as mpl
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import cm
from matplotlib import pyplot as plt
from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import (
    MaxAbsScaler,
    MinMaxScaler,
    Normalizer,
    PowerTransformer,
    QuantileTransformer,
    RobustScaler,
    StandardScaler,
    minmax_scale,
)

sns.set(style="white", color_codes=True)

plt.figure(figsize=(15, 5))
seed = 42
np.random.seed(seed)



<!-- #region id="88ac5d01" -->
Il arrive souvent que les valeurs des variables $x_{i}$ dans les bases de données diffèrent de
plusieurs ordres de grandeur. Beaucoup d'algorithmes d'apprentissage sont sensibles
à ce phénomène. On transforme alors les données afin de les ramener dans des
intervalles de valeurs pour lesquels certaines caractéristiques de la distribution (moyenne, médiane,
minimum, maximum, etc.) deviennent similaires. Ces opérations sont réalisées par normalisation et
par transformation non linéaire des données. Nous allons en voir plusieurs exemples dans ce module.
<!-- #endregion -->

<!-- #region id="e357d7ae" -->
# <a id=importance-de-la-normalisation-des-données>Importance de la normalisation des données</a>
<!-- #endregion -->

<!-- #region colab_type="text" id="f0a07a3e" -->
La normalisation des données est une des opérations les plus importantes en préparation des données.
Pour en donner une idée, supposons qu'on utilise le modèle de régression suivant pour prédire une réponse
$y$ en fonction de deux variables $x_{1}$ et $x_{2}$.

$$y \approx a_{0}+a_{1}x_{1}+a_{2}x_{2}$$

Si la variable $x_{1} \in [0, 1]$ et la variable $x_{2} \in [0, 1000]$, le dernier terme de
l'équation précédente aura le plus d'impact sur la solution de la régression. Cela ne veut pas dire
qu'il est le plus important. Par exemple si $x_{1}$ correspond à une distance mesurée en mètre et $x_{2}$ correspond
à une autre distance mesurée en millimètres, alors tous les deux correspondent au même intervalle de longueur. Ils ont une
importance égale si l'on tient compte des échelles respectives.

La normalisation enlève les échelles et met les deux variables normalisées $\tilde{x}_{1}$ et $\tilde{x}_{2}$
sur un même pied d'égalité. Si l'on effectue la régression avec le modèle utilisant les variables normalisées,
on obtient

$$y \approx a_{0}+a_{1}\tilde{x}_{1}+a_{2}\tilde{x}_{2}.$$

Dans ce cas-ci, les valeurs estimées des coefficients permettront de bien comparer entre elles les variables.
Par exemple, si $a_{1} \gg a_{2}$ alors la variable $x_{1}$ a un plus grand effet sur la réponse $y$ que la
variable $x_{2}$. La normalisation permet une comparaison non biaisée des variables entre elles en plus de permettre
aux coefficients de prendre toute leur importance. Il y a d'autres raisons justifiant l'étape de normalisation,
mais celles-ci sont les plus importantes à retenir.

<!-- #endregion -->

<!-- #region id="f83e6594" -->
# <a id=rééchelonnage-ou-normalisation>Rééchelonnage ou normalisation</a>
<!-- #endregion -->

<!-- #region id="d9ace578" -->
La séquence suivante est inspirée des [exemples de code](https://scikit-learn.org/stable/auto_examples/preprocessing/plot_all_scaling.html#sphx-glr-auto-examples-preprocessing-plot-all-scaling-py) de la librairie Scikit-learn.
<!-- #endregion -->

<!-- #region id="b3a4ee50" -->
Le code ci-dessous sert à afficher les graphiques joliment.
<!-- #endregion -->



In [ ]:
cmap = getattr(cm, "jet_r", cm.jet_r)


def create_axes(title, figsize=(16, 6)):
    fig = plt.figure(figsize=figsize)
    fig.suptitle(title)

    # Définition des axes pour la première figure.
    left, width = 0.1, 0.22
    bottom, height = 0.1, 0.7
    bottom_h = height + 0.15
    left_h = left + width + 0.02

    rect_scatter = [left, bottom, width, height]
    rect_histx = [left, bottom_h, width, 0.1]
    rect_histy = [left_h, bottom, 0.05, height]

    ax_scatter = plt.axes(rect_scatter)
    ax_histx = plt.axes(rect_histx)
    ax_histy = plt.axes(rect_histy)

    # Définition des axes pour la zone agrandie.
    left = width + left + 0.2
    left_h = left + width + 0.02

    rect_scatter = [left, bottom, width, height]
    rect_histx = [left, bottom_h, width, 0.1]
    rect_histy = [left_h, bottom, 0.05, height]

    ax_scatter_zoom = plt.axes(rect_scatter)
    ax_histx_zoom = plt.axes(rect_histx)
    ax_histy_zoom = plt.axes(rect_histy)

    # Définition des axes pour la barre de couleurs.
    left, width = width + left + 0.13, 0.01

    rect_colorbar = [left, bottom, width, height]
    ax_colorbar = plt.axes(rect_colorbar)

    return (
        (ax_scatter, ax_histy, ax_histx),
        (ax_scatter_zoom, ax_histy_zoom, ax_histx_zoom),
        ax_colorbar,
    )


def plot_distribution(axes, X, y, hist_nbins=50, title="", x0_label="", x1_label=""):
    ax, hist_X1, hist_X0 = axes

    ax.set_title(title)
    ax.set_xlabel(x0_label)
    ax.set_ylabel(x1_label)

    # Diagramme de points
    colors = cmap(y)
    ax.scatter(X[:, 0], X[:, 1], alpha=0.5, marker="o", s=5, lw=0, c=colors)

    # Histogramme pour l'axe X1 (feature 5)
    hist_X1.set_ylim(ax.get_ylim())
    hist_X1.hist(
        X[:, 1],
        bins=hist_nbins,
        orientation="horizontal",
        color="grey",
        ec="grey",
    )
    hist_X1.axis("off")

    # Histogramme pour l'axe X0 (feature 0)
    hist_X0.set_xlim(ax.get_xlim())
    hist_X0.hist(
        X[:, 0],
        bins=hist_nbins,
        orientation="vertical",
        color="grey",
        ec="grey",
    )
    hist_X0.axis("off")


def make_plot(title, X):
    ax_zoom_out, ax_zoom_in, ax_colorbar = create_axes(title)
    axarr = (ax_zoom_out, ax_zoom_in)
    plot_distribution(
        axarr[0],
        X,
        y,
        hist_nbins=200,
        x0_label="Revenu médian",
        x1_label="Nombre d'occupants",
        title="Données complètes",
    )

    # Élimination des outliers
    zoom_in_percentile_range = (0, 99)
    cutoffs_X0 = np.percentile(X[:, 0], zoom_in_percentile_range)
    cutoffs_X1 = np.percentile(X[:, 1], zoom_in_percentile_range)

    non_outliers_mask = np.all(X > [cutoffs_X0[0], cutoffs_X1[0]], axis=1) & np.all(
        X < [cutoffs_X0[1], cutoffs_X1[1]], axis=1
    )
    plot_distribution(
        axarr[1],
        X[non_outliers_mask],
        y[non_outliers_mask],
        hist_nbins=200,
        x0_label="Revenu médian",
        x1_label="Nombre d'occupants",
        title="Sans outliers",
    )

    norm = mpl.colors.Normalize(y_full.min(), y_full.max())
    mpl.colorbar.ColorbarBase(
        ax_colorbar,
        cmap=cmap,
        norm=norm,
        orientation="vertical",
        label="Valeur de la maison *100 K$",
    )



<!-- #region id="80bdf6bb" -->
## <a id=lecture-de-la-base-de-données>Lecture de la base de données</a>
<!-- #endregion -->

<!-- #region id="c5c9cc31" -->
<p>&nbsp;</p>
<div align="center">
    <img src= "../images/house-image.jpeg"  width="400" />
    <div>
    <font size="0.5">Image Source: https://pxhere.com/cs/photo/672220</font>
    </div>
</div>
<p>&nbsp;</p>
<!-- #endregion -->

<!-- #region colab_type="text" id="a214cffc" -->
Celle-ci contient huit caractéristiques mesurées pour environ 21 000 zones d'habitation (*blocks*) en Californie.
Elle a été conçue afin de prédire, entre autres, la valeur médiane des maisons en centaines de milliers
de dollars US (en 1990).
<!-- #endregion -->



In [ ]:
dataset = fetch_california_housing()
X_full, y_full = dataset.data, dataset.target

# Description de la base de données
print(dataset.DESCR)



<!-- #region colab_type="text" id="a214cffc" -->
## <a id=méthodes-de-transformation>Méthodes de transformation</a>
<!-- #endregion -->

<!-- #region id="cc80ade5" -->
### <a id=données-brutes>Données brutes</a>
<!-- #endregion -->

<!-- #region id="614603ab" -->
Dans les exemples qui suivent, on va comparer les distributions de deux caractéristiques.
La caractéristique « $0$ », le revenu médian dans une zone, a une distribution
à queue lourde (*heavy-tail distribution*). La caractéristique « $5$ », le nombre d'occupants dans
la même zone, contient quelques données aberrantes (*outliers*) très prononcées.

Toutes les figures qui suivent utilisent la convention suivante:


- chaque panneau montre les valeurs de $(x_{0}, x_{5})$ avec un code de couleur correspondant au
prix médian des maisons dans chaque zone,
- le panneau de gauche montre les données transformées contenant des valeurs aberrantes,
- le panneau de droite montre les données transformées et nettoyées des valeurs aberrantes.
<!-- #endregion -->



In [ ]:
X = X_full[:, [0, 5]]

# On rééchelonne la donnée de sortie, le prix médian des maisons, entre 0 et 1 pour mieux visualiser la
# légende (étalement de la couleur).
y = minmax_scale(y_full)

data = ("Unscaled data", X)
make_plot(*data)



<!-- #region id="32e1e43d" -->
On remarque que la présence de valeurs aberrantes dans le panneau de gauche écrase les distributions
de données et les histogrammes.
<!-- #endregion -->

<!-- #region colab_type="text" id="216eaf3f" -->
### <a id=standard>Standard</a>
<!-- #endregion -->

<!-- #region id="3ece8b88" -->
La normalisation standard d'une variable $x_i$ se calcule comme suit

$$x_i = \dfrac{x_i - \bar{x}}{\sigma(x)}$$

Elle consiste à enlever la moyenne de la variable et diviser le résultat par son écart-type. Elle permet de transformer
une distribution normale/gaussienne $N(\mu, \sigma)$ en une autre $N(0,1)$ de moyenne nulle et de variance unité. Autrement, elle ne change pas la forme de la distribution des données. C'est la transformation la plus utilisée.

Elle ne change rien à l'affichage puisque les fonctions d'affichage ajustent par défaut les bornes des figures afin de contenir l'ensemble des données. L'aspect d'une distribution de points ne change donc pas avant et après la transformation
mis à part les bornes des axes.

<!-- #endregion -->



In [ ]:
data = ("Data after standard scaling", StandardScaler().fit_transform(X))
make_plot(*data)



<!-- #region colab_type="text" id="59583029" -->
### <a id=min-max>Min-max</a>
<!-- #endregion -->

<!-- #region id="da872a92" -->
Cette transformation est la plus utilisée après celle de la normalisation standard. Elle force les valeurs de la
variable $x_i$ à être distribuées entre 0 et 1. La variable transformée n'a pas nécessairement une valeur moyenne de 0 comme dans le cas précédent. Elle est aussi plus sensible aux valeurs aberrantes, car celles-ci déterminent
les valeurs minimale et maximale de la variable qui sont utilisées dans la transformation.

$$x_i = \dfrac{x_i - \min_i{x_i}}{\max_i{x_i}-\min_i{x_i}}$$


C'est une autre transformation linéaire. Les résultats sont similaires aux précédents, mis à part les bornes des axes.
<!-- #endregion -->



In [ ]:
data = ("Data after min-max scaling", MinMaxScaler().fit_transform(X))
make_plot(*data)



<!-- #region colab_type="text" id="b961d2c4" -->
### <a id=max-abs>Max-abs</a>
<!-- #endregion -->

<!-- #region id="587a4d09" -->
Cette transformation impose aux données une borne valant $-1$ si $\max_i{x_i}<0$ ou $+1$ si $\max_i{x_i}>0$.

$$x_i = \dfrac{x_i}{\max_i{|x_i|}}$$

C'est une autre transformation linéaire. Les résultats sont similaires aux précédents, mis à part les bornes des axes.
<!-- #endregion -->



In [ ]:
data = ("Data after max-abs scaling", MaxAbsScaler().fit_transform(X))
make_plot(*data)



<!-- #region colab_type="text" id="ea9b5565" -->
### <a id=robuste-basé-sur-les-quantiles-1-2-et-3>Robuste (basé sur les quantiles 1, 2 et 3)</a>
<!-- #endregion -->

<!-- #region id="32df4e47" -->
Cette transformation ressemble à celle la normalisation standard, où l'on a remplacé la moyenne et l'écart-type de la distribution des valeurs de la variable $x_i$ par des estimateurs plus robustes; la médiane $Q_2(x)$ et l'intervalle interquartile $Q_3(x) - Q_1(x)$.

$$x_i = \dfrac{x_i - Q_2(x)}{Q_3(x) - Q_1(x)}$$

La moyenne et l'écart-type sont des estimateurs statistiques sensibles aux valeurs aberrantes. Dans ces situations, la médiane et l’intervalle interquartile donnent souvent de meilleurs résultats. La transformation résultante demeure quand même linéaire; mêmes résultats que précédemment, aux bornes près!

<!-- #endregion -->



In [ ]:
data = (
    "Data after robust scaling",
    RobustScaler(quantile_range=(25, 75)).fit_transform(X),
)
make_plot(*data)



<!-- #region colab_type="text" id="60b532fc" -->
### <a id=transformation-par-puissance>Transformation par puissance</a>
<!-- #endregion -->

<!-- #region id="be749ca0" -->
Il existe plusieurs familles de transformations non linéaires qui permettent de transformer
une distribution arbitraire de données en une autre approximativement gaussienne. C'est très utile
pour traiter les problèmes liés à l'hétéroscédasticité (variance non constante) ou ceux pour lesquels
une distribution normale des données est souhaitée. En effet, plusieurs méthodes en apprentissage automatique ont été
développées en faisant l'hypothèse de normalité dans les données à traiter. Lorsque ce n'est pas le cas,
elles ne performent pas très bien.

Une transformation non linéaire fausse les corrélations linéaires entre les variables mesurées à la même échelle.
Toutefois, elle rend les variables mesurées à différentes échelles plus directement comparables.

L'exemple suivant utilise deux familles de transformations: Yeo-Johnson et Box-Cox. Ce sont des cas particuliers de la [transformation par puissance](https://en.wikipedia.org/wiki/Power_transform).

<!-- #endregion -->



In [ ]:
data = (
    "Données après transformation par puissance (Yeo-Johnson)",
    PowerTransformer(method="yeo-johnson").fit_transform(X),
)
make_plot(*data)

data = (
    "Données après transformation par puissance (Box-Cox)",
    PowerTransformer(method="box-cox").fit_transform(X),
)
make_plot(*data)



<!-- #region id="d5834e5f" -->
Comparez ces résultats avec les précédents. Les distributions des variables transformées sont devenues presque gaussiennes.
L'asymétrie des distributions précédentes a disparu.
<!-- #endregion -->

<!-- #region colab_type="text" id="845ba7bf" -->
### <a id=transformation-quantile-à-quantile>Transformation quantile à quantile</a>
<!-- #endregion -->

<!-- #region id="fcbe9d1a" -->
La fonction de densité cumulée d'une variable est utilisée pour projeter les valeurs originales
(à l'intérieur d'une plage de valeurs) vers de nouvelles valeurs ayant une distribution précise
(gaussienne, uniforme, etc.). Les valeurs originales hors de la plage seront projetées aux limites
de la distribution en sortie. Voilà une autre transformation non linéaire.

Voyez cet [exemple interactif](https://geostatisticslessons.com/lessons/normalscore)
où l'on transforme une variable $z$ ayant une distribution quelconque en une autre $y$ ayant
une distribution gaussienne de moyenne $0$ et de variance $1$.

L'exemple suivant transforme les distributions originales en distributions gaussiennes et uniformes.
<!-- #endregion -->



In [ ]:
data = (
    "Données après transformation par quantile" "(distribution finale gaussienne)",
    QuantileTransformer(output_distribution="normal").fit_transform(X),
)
make_plot(*data)

data = (
    "Données après transformation par quantile" "(distribution finale uniforme)",
    QuantileTransformer(output_distribution="uniform").fit_transform(X),
)
make_plot(*data)



<!-- #region id="43c92943" -->
Notez à quel point les distributions finales sont presque parfaitement gaussiennes et uniformes! Cette
approche fonctionne bien avec un grand nombre de données. Ce type de transformation est non
linéaire et peut contenir des discontinuités. C'est un avantage sur la méthode précédente qui est
continue, par définition.
<!-- #endregion -->

<!-- #region colab_type="text" id="62f5c75e" -->
### <a id=normalisation-vectorielle>Normalisation vectorielle</a>
<!-- #endregion -->

<!-- #region id="3d451ab1" -->
Cette transformation est totalement différente des précédentes. Alors que ces dernières étaient appliquées une variable
$x_i$ à la fois, celle-ci utilise la norme du vecteur $X$ en entier pour normaliser toutes les variables $x_i$ en même temps.

$$x_i = \dfrac{x_i}{ ||X|| }$$


La mise à l'échelle des entrées, au moyen de la norme du vecteur d'entrées, est une opération courante pour la classification de texte et le regroupement des données. La figure suivante montre l'effet de la normalisation d'échelle avec des données non reliées à l'analyse de texte; c'est l'exemple des maisons californiennes.
<!-- #endregion -->



In [ ]:
data = (
    "Données après transformation par normalisation L2",
    Normalizer().fit_transform(X),
)
make_plot(*data)



<!-- #region id="6c28e1d4" -->
Comment interpréter ces résultats si différents des précédents? On remarque que la transformation.

$$x_i = \frac{x_i}{ ||x_i|| }$$

positionne chaque vecteur $x_i$ sur un cercle de rayon 1, dans ce cas-ci un quart de cercle puisque
les composantes du vecteur sont positives.

Quel avantage y a-t-il à faire cela? Notez que la couleur change du rouge au bleu de façon assez uniforme. Ainsi,
le prix des maisons dans une zone augmente lorsque le revenu médian des résidents augmente et que leur
nombre diminue. L'extrémité bleue doit correspondre au quartier de Beverly Hills, à Los Angeles, et l'extrémité
rouge doit correspondre à un quartier défavorisé dans la même mégalopole.

<!-- #endregion -->

<!-- #region colab_type="text" id="c7a0158a" -->
## <a id=comparaison-entre-les-différents-rééchantillonneurs>Comparaison entre les différents rééchantillonneurs</a>
<!-- #endregion -->

<!-- #region id="5ff86c57" -->
Comment comparer entre elles les différentes méthodes de transformation? Examinons comment chacune
affecte une même caractéristique $x_i$, dans ce cas-ci le revenu médian des résidents dans chaque zone.
<!-- #endregion -->



In [ ]:
dfX = pd.DataFrame(dataset.data, columns=dataset.feature_names)
dfX = dfX.drop(dataset.feature_names[1:], axis=1)

col = dfX.MedInc.values.reshape(-1, 1)

scalers = [
    ("Standard", StandardScaler()),
    ("Min-max", MinMaxScaler()),
    ("Max-abs", MaxAbsScaler()),
    ("Robuste", RobustScaler(quantile_range=(25, 75))),
    ("Quantile (gaussian)", QuantileTransformer(output_distribution="normal")),
    ("Quantile (uniforme)", QuantileTransformer(output_distribution="uniform")),
]

for scaler in scalers:
    dfX[scaler[0]] = scaler[1].fit_transform(col)

dfX.describe()



<!-- #region id="92730133" -->
Pour mieux comprendre les différences entre les rééchantillonneurs, la figure suivante montre la distribution de la
caractéristique `Revenu médian` avant et après chaque type de transformation.
<!-- #endregion -->



In [ ]:
cpt = 1
f = plt.figure(figsize=(30, 12))
for scaler in scalers:
    name = scaler[0]
    ax = f.add_subplot(2, len(scalers), cpt)

    sns.histplot(dfX.MedInc, ax=ax, kde=True, color="b")
    ax.axvline(dfX.MedInc.mean(), color="k", linestyle="dashed", linewidth=1)
    sns.histplot(dfX[name], ax=ax, kde=True, color="r")
    ax.axvline(dfX[name].mean(), color="k", linestyle="dashed", linewidth=1)
    ax.set_title(name)

    ax = f.add_subplot(2, len(scalers), cpt + len(scalers))
    g = sns.regplot(x="MedInc", y=name, data=dfX, ax=ax)
    cpt += 1



<!-- #region id="6d2c801b" -->
Dans la ligne du haut, les distributions des valeurs originales apparaissent en bleu, et celles des valeurs
transformées, en rouge. La ligne du bas montre la relation entre le `Revenu médian` original et sa valeur transformée. Les quatre
premières transformations sont linéaires et les deux suivantes sont non linéaires.
<!-- #endregion -->

<!-- #region id="6946a633" -->
# <a id=le-traitement-des-variables-catégorielles>Le traitement des variables catégorielles</a>
<!-- #endregion -->

<!-- #region id="80f64164" -->
<p>&nbsp;</p>
<div align="center">
    <img src= "../images/plastic-balls.jpeg"  width="500" />
    <div>
    <font size="0.5">Image Source: https://pixabay.com/sv/photos/bollhav-m%c3%a5ngf%c3%a4rgad-bollar-leksak-1029865/</font>
    </div>
</div>
<p>&nbsp;</p>
<!-- #endregion -->

<!-- #region id="18f38646" -->
Il est rare qu'un projet d'analyse des données
n'utilise que des variables numériques continues (température, salaire, etc.). En pratique,
il faut aussi tenir compte de variables catégorielles (sexe, couleur, etc.) et ordinales
(niveau d'appréciation, échelle d'intensité des piments, etc.). On s'intéresse ici aux variables catégorielles.

Comment effectuer une régression si une des variables d'entrée prend les valeurs
$\{'\text{Rouge}', '\text{Vert}', '\text{Bleu}', '\text{Jaune}'\}$? Peut-on la convertir en une variable entière prenant les
valeurs $\{0, 1, 2, 3\}$ afin de l'intégrer dans les calculs? Mais alors, puisque $3>2$ est-ce que cela implique que
la couleur jaune est plus importante que la bleue? Bien sûr que non. Le traitement des variables catégorielles
gère ce genre de situation.

Nous allons voir dans ce qui suit quelques exemples de traitement de variables catégorielles
souvent utilisés en pratique. 

> À noter que plusieurs d'entre eux ont été inspirés de l'aide mémoire en prétraitement des données d'
[Elisabeth Reitmayr](https://github.com/lis365b/data_analysis/blob/master/data_prep_python_cheatsheet.md).
Jetez un coup d'oeil pour y apprendre plein de trucs utiles.

Dans les exemples qui suivent, on suppose que des données ont été enregistrées dans un tableau de données 
`df` de la librairie Pandas. Cette librairie, ainsi que d'autres en Python, sont décrites dans un autre
module de la formation. En bref, un tableau de données est un moyen de représenter et de travailler avec des données tabulaires de divers types. Il peut être considéré comme un tableau qui organise les données en lignes et en colonnes, ce qui en fait une structure de données bidimensionnelle.

<!-- #endregion -->

<!-- #region id="15630ac6" -->
## <a id=regroupement-de-catégories-en-une-seule-valeur>Regroupement de catégories en une seule valeur</a>
<!-- #endregion -->

<!-- #region id="4b55f5c3" -->
Par exemple, on a une variable catégorielle qui contient des noms de pays. On veut créer une nouvelle variable « `'Filtre'` » qui
ne garde que les trois pays les plus populeux et place les autres dans une classe « `'autre'` ».
<!-- #endregion -->



In [ ]:
# Fonction de remplacement
def repl(x):
    if x == "USA":
        return "USA"
    elif x == "Chine":
        return "Chine"
    elif x == "Inde":
        return "Inde"
    else:
        return "Autre"


df = pd.DataFrame(
    ["Canada", "USA", "France", "Chine", "Burundi", "UK", "Brésil", "Inde"],
    columns=["Pays"],
)

df["Filtre"] = df["Pays"].apply(repl)

df.head(10)



<!-- #region id="4a229698" -->
## <a id=regroupement-de-variables-numériques-en-catégories>Regroupement de variables numériques en catégories</a>
<!-- #endregion -->

<!-- #region id="8bc6a9b9" -->
### <a id=catégories-de-taille-égales>Catégories de taille égales</a>
<!-- #endregion -->

<!-- #region id="c8d5a5e7" -->
Par exemple, on veut convertir les notes d'un examen en trois classes $\{\text{'Faible'}, \text{'Moyenne' et } \text{'Forte'}\}$ réparties également entre la note la plus faible et la plus forte.
<!-- #endregion -->



In [ ]:
df = pd.DataFrame([60, 50, 90, 90, 100, 40, 30, 60, 40, 70], columns=["Note"])

df["Évaluation"] = pd.qcut(df["Note"], q=3, labels=["Faible", "Moyenne", "Forte"])

df.head(10)



<!-- #region id="59deffc4" -->
### <a id=catégories-par-intervalles-identiques>Catégories par intervalles identiques</a>
<!-- #endregion -->

<!-- #region id="342e76d7" -->
Par exemple, on veut convertir des valeurs de température en cinq classes $\{\text{'Très froid'}, \text{'Froid'}, \text{'Tiède'}, \text{'Chaud'}, \text{'Très chaud'}\}$ réparties dans les intervalles $[0,20[, [20,40[,\cdots$
<!-- #endregion -->



In [ ]:
df = pd.DataFrame([60, 50, 90, 15, 10, 45, 30, 60, 40, 75], columns=["Température"])

bins = [0, 20, 40, 60, 80, 100]
df["Impression"] = pd.cut(
    df["Température"],
    bins,
    labels=["Très froid", "Froid", "Tiède", "Chaud", "Très chaud"],
)

df.head(10)



<!-- #region id="0075b56c" -->
## <a id=encoder-des-variables-catégorielles>Encoder des variables catégorielles</a>
<!-- #endregion -->

<!-- #region id="d1709424" -->
### <a id=encoder-une-variable-booléenne-en-la-transformant-en-entier>Encoder une variable booléenne en la transformant en entier</a>
<!-- #endregion -->

<!-- #region id="1b000784" -->
Par exemple, on veut convertir la variable `Sexe` contenant initialement les valeurs $\{\text{'Femme'}, \text{'Homme'}\}$ en entiers $\{0, 1\}$ correspondants.
<!-- #endregion -->



In [ ]:
df = pd.DataFrame(
    ["Homme", "Homme", "Femme", "Femme", "Femme", "Homme", "Femme", "Femme"],
    columns=["Sexe"],
)

df["Genre"] = (df["Sexe"] == "Homme").astype(int)

df.head(10)



<!-- #region id="e41f1911" -->
### <a id=encoder-manuellement-en-établissant-les-correspondances-avec-un-dictionnaire>Encoder manuellement en établissant les correspondances avec un dictionnaire</a>
<!-- #endregion -->

<!-- #region id="1618bf94" -->
Autre façon d'effectuer la même conversion.
<!-- #endregion -->



In [ ]:
dic = {"Femme": 0, "Homme": 1}
df["Genre"] = df["Sexe"].map(dic)
df.head(10)



<!-- #region id="8dc6997e" -->
### <a id=encoder-automatiquement>Encoder automatiquement</a>
<!-- #endregion -->

<!-- #region id="6f5fb49a" -->
Par exemple, supposons qu'une variable « `'COL'` » contienne cinq catégories $\{\text{'A'}, \text{'B'}, \text{'C'}, \text{'D'}, \text{'E'}\}$. On veut la
transformer en cinq nouvelles variables, dont chacune prendra les valeurs $\{0, 1\}$ selon que $\text{COL}=\text{'A'}$ ou non,
et ainsi de suite pour les autres valeurs de la variable « `'COL'` ». Initialisons d'abord la variable:
<!-- #endregion -->



In [ ]:
df = pd.DataFrame(["A", "A", "B", "C", "B", "E", "D"], columns=["COL"])
df.head(10)



<!-- #region id="9ee9d0bf" -->
Transformons la variable « `'COL'` » en cinq variables factices (*dummy variables*). On peut utiliser la fonction
[`get_dummies`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.get_dummies.html) de Pandas
ou encore la classe [`OneHotEncoder`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html) de Scikit-learn. Utilisons la première.
<!-- #endregion -->



In [ ]:
df2 = pd.get_dummies(df["COL"])
df2.head(10)



<!-- #region id="4a3351f5" -->
Après ces étapes, il y a maintenant autant de colonnes dans le tableau de données `df2` que de
catégories dans la colonne « `'COL'` ». Ces colonnes sont colinéaires; n'importe laquelle d'entre elles
peut être déduite des quatre autres.

Si nécessaire, on peut éliminer la colinéarité en laissant tomber la première colonne pour ne conserver que les quatre autres en faisant ceci:
<!-- #endregion -->



In [ ]:
df2 = pd.get_dummies(df["COL"], drop_first=True)
df2.head(10)



<!-- #region id="ecd7f568" -->
Cette opération peut être effectuée avec toutes les variables catégorielles d'un tableau de données. Regardons un autre exemple ou le tableau de données `df` contient maintenant une variable entière et deux catégorielles.
<!-- #endregion -->



In [ ]:
df = pd.DataFrame(
    {
        "Sexe": ["H", "F", "F", "H"],
        "Ville": ["Québec", "Québec", "Montréal", "Laval"],
        "Age": [20, 50, 30, 10],
    }
)
df.head()



<!-- #region id="111ff07c" -->
Convertissons les deux variables catégorielles en variables factices. Notez que le nom de chaque
variable catégorielle est maintenant utilisé comme préfixe afin de ne pas confondre les variables factices entre elles!
<!-- #endregion -->



In [ ]:
df_2 = pd.get_dummies(df)
df_2.head()



<!-- #region id="73ff4be6" -->
Si nécessaire, on peut à nouveau éliminer la colinéarité en laissant tomber la première variable factice de chaque variable catégorielle:
<!-- #endregion -->



In [ ]:
df_2 = pd.get_dummies(df, drop_first=True)
df_2.head()
